### 金庸小说全集———人名embedding可视化分析

### 0.环境准备

In [1]:
# 导入所需的库
import os
import jieba
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import numpy as np
from collections import Counter,defaultdict
from sklearn.decomposition import PCA
import plotly.graph_objects as go
import plotly.express as px
from tqdm import tqdm

print("环境准备完成")
print(f"PyTorch版本：{torch.__version__}")

环境准备完成
PyTorch版本：2.2.1


### 1.数据加载与预处理

In [2]:
# 文件路径
data_dir = 'jinyong_all_novel'
stopwords_path = 'stopwords.txt'
person_names_path = 'jinyong_all_person.txt'

# 数据读取
# 读取停用词
with open(stopwords_path, 'r', encoding='utf-8') as f:
    stopwords = set([line.strip() for line in f])

# 读取全人物列表
with open(person_names_path, 'r', encoding='utf-8') as f:
    all_persons = [line.strip() for line in f]

# 数据预处理
# 初始化分词器
jieba.initialize()
# 添加人物词典
for person in all_persons:
    jieba.add_word(person, freq=100000)  # 高频保证准确切分人名

# 逐本分词处理
book_segments = {}
full_corpus = []
person_books = defaultdict(list)  # 记录人物出现书籍

print("开始分词...")
for filename in tqdm(os.listdir(data_dir)):
    if filename.endswith('.txt'):
        # 读取文件
        book_name = filename[:-4]
        with open(os.path.join(data_dir, filename), 'r', encoding='utf-8') as f:
            text = f.read()
        
        # 分词处理
        words = jieba.lcut(text)
        # 清洗过滤
        filtered = [
            w for w in words 
            if w not in stopwords 
            and len(w) > 1 
            and not w.isspace()
        ]
        
        # 存储结果
        book_segments[book_name] = filtered
        full_corpus.extend(filtered)

        # 记录本书中出现的人物
        current_persons = set()
        for w in filtered:
            if w in all_persons:
                current_persons.add(w)
        for person in current_persons:
            person_books[person].append(book_name)
        
        # 输出统计信息
        print(f"\n《{book_name}》分词完成")
        print(f"有效词汇量：{len(filtered):,}")
        print("前20个分词示例：", filtered[:20])

print("\n全部小说处理完成！")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ASUS\AppData\Local\Temp\jieba.cache
Loading model cost 0.613 seconds.
Prefix dict has been built successfully.


开始分词...


  7%|▋         | 1/15 [00:04<00:58,  4.15s/it]


《金庸-1955-书剑恩仇录》分词完成
有效词汇量：133,161
前20个分词示例： ['书剑', '恩仇录', '第一回', '古道', '腾驹', '白发', '危峦', '击剑', '识青翎', '清乾隆', '十八年', '六月', '陕西', '扶风', '延绥镇', '总兵', '衙门', '内院', '一个十四岁', '女孩儿']


 13%|█▎        | 2/15 [00:07<00:49,  3.81s/it]

...


《金庸-1967-笑傲江湖》分词完成
有效词汇量：240,939
前20个分词示例： ['笑傲江湖', '灭门', '风熏', '花香', '醉人', '南国', '春光', '漫烂', '季节', '福建省', '福州', '西门', '大街', '青石板路', '笔直', '伸展出去', '直通', '西门', '一座', '建构']


100%|██████████| 15/15 [01:18<00:00,  5.29s/it]


《金庸-1969—1972-鹿鼎记》分词完成
有效词汇量：296,257
前20个分词示例： ['鹿鼎记', '第一回', '纵横', '清流', '峭茜', '风期', '月旦评', '北风', '如刀', '满地', '冰霜', '江南', '海滨', '一条', '路上', '一队', '清兵', '手执', '刀枪', '七辆']

《金庸-1970-越女剑》分词完成
有效词汇量：4,550
前20个分词示例： ['越女剑', '两名', '剑士', '倒转', '剑尖', '右手', '剑柄', '左手', '搭于', '右手', '手背', '躬身行礼', '两人', '身子', '尚未', '白光闪', '跟着', '声响', '双剑相', '两人']


100%|██████████| 15/15 [01:18<00:00,  5.23s/it]


全部小说处理完成！


### 2.构建训练数据集

In [3]:
# 构建词汇表
MIN_FREQ = 5  # 最小词频

# 统计词频
word_counts = Counter(full_corpus)
vocab = [word for word, count in word_counts.items() if count >= MIN_FREQ]
vocab_size = len(vocab)

# 创建映射字典
word2idx = {word:i for i, word in enumerate(vocab)}
idx2word = {i:word for i, word in enumerate(vocab)}

print(f"\n总词汇量（词频≥{MIN_FREQ}）：{vocab_size:,}")

# 准备训练数据
# 数据集定义
WINDOW_SIZE = 5
BATCH_SIZE = 2048
EMBEDDING_DIM = 128

class SkipGramDataset(data.Dataset):
    def __init__(self, corpus, word2idx, window_size):
        self.data = [word2idx[word] for word in corpus if word in word2idx]
        self.window_size = window_size
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        center_word = self.data[idx]
        # 随机选择上下文窗口内的一个词
        start = max(0, idx - self.window_size)
        end = min(len(self.data), idx + self.window_size + 1)
        context_words = self.data[start:idx] + self.data[idx+1:end]
        context_word = np.random.choice(context_words) if context_words else center_word
        return torch.LongTensor([center_word]), torch.LongTensor([context_word])

# 创建数据加载器
dataset = SkipGramDataset(full_corpus, word2idx, WINDOW_SIZE)
dataloader = data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)


总词汇量（词频≥5）：42,930


### 3.训练Word2Vec模型

In [4]:
# 模型定义
class Word2Vec(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embed_dim)
        self.linear = nn.Linear(embed_dim, vocab_size, bias=False)
    
    def forward(self, center_words):
        embeds = self.embeddings(center_words)   # 获取嵌入向量
        out = self.linear(embeds)            # 将嵌入向量通过线性层映射到输出
        return out

# 初始化模型，优化器和损失函数
model = Word2Vec(vocab_size, EMBEDDING_DIM)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

# 模型训练
NUM_EPOCHS = 10  # 训练轮数
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print("\n开始训练...")
# 训练过程
for epoch in range(NUM_EPOCHS):
    total_loss = 0
    pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")
    
    for centers, contexts in pbar:
        centers = centers.squeeze().to(device)
        contexts = contexts.squeeze().to(device)
        
        optimizer.zero_grad()
        outputs = model(centers)
        loss = loss_fn(outputs, contexts)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        pbar.set_postfix(loss=loss.item())
    
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1} 平均损失: {avg_loss:.4f}")

print("\n训练完成！")

# 保存模型
print("\n保存模型中...")
torch.save({
    'model_state_dict': model.state_dict(),
    'word2idx': word2idx,
    'vocab': vocab
}, 'jinyong_word2vec.pth')
print("模型保存完成！")


开始训练...


Epoch 1/10: 100%|██████████| 983/983 [23:45<00:00,  1.45s/it, loss=9.84]


Epoch 1 平均损失: 10.3890


Epoch 2/10: 100%|██████████| 983/983 [25:39<00:00,  1.57s/it, loss=9.42]


Epoch 2 平均损失: 9.5808


...

Epoch 8 平均损失: 9.0520


Epoch 9/10: 100%|██████████| 983/983 [22:25<00:00,  1.37s/it, loss=9.05]


Epoch 9 平均损失: 9.0184


Epoch 10/10: 100%|██████████| 983/983 [21:34<00:00,  1.32s/it, loss=9.06]


Epoch 10 平均损失: 8.9935

训练完成！

保存模型中...
模型保存完成！


### 4.提取人物向量和PCA降维

In [5]:
# 提取人物向量
valid_persons = []
embeddings = []
book_labels = []

for person in all_persons:
    if person in word2idx:
        valid_persons.append(person)
        emb = model.embeddings.weight.data[word2idx[person]].cpu().numpy()
        embeddings.append(emb)

        # 获取人物主要出现书籍
        books = person_books.get(person, [])
        if books:
            # 取出现次数最多的书籍
            book_counter = Counter(books)
            main_book = book_counter.most_common(1)[0][0]
        else:
            main_book = "未知"
        book_labels.append(main_book)

embeddings = np.array(embeddings)
print(f"\n成功提取 {len(valid_persons)}/{len(all_persons)} 个人物向量")

# PCA降维
# 二维降维
pca_2d = PCA(n_components=2)
emb_2d = pca_2d.fit_transform(embeddings)

# 三维降维
pca_3d = PCA(n_components=3)
emb_3d = pca_3d.fit_transform(embeddings)

print("\n降维完成，降维结果方差解释率：")
print("2D:", pca_2d.explained_variance_ratio_)
print("3D:", pca_3d.explained_variance_ratio_)


成功提取 1179/1411 个人物向量

降维完成，降维结果方差解释率：
2D: [0.01794518 0.01551869]
3D: [0.01794431 0.01574976 0.01429256]


### 5.交互式可视化

In [8]:
# 2D可视化
fig_2d = px.scatter(
    x=emb_2d[:,0], y=emb_2d[:,1],
    color=book_labels,
    text=valid_persons,
    title="金庸人物关系二维投影",
    labels={'x': 'PC1', 'y': 'PC2'},
    hover_name=valid_persons,
    hover_data={'所属书籍': book_labels}
)
fig_2d.update_traces(
    marker=dict(size=10, opacity=0.8),
    textfont=dict(size=10),
    textposition='top center'
)
fig_2d.write_html("person_2d.html")
fig_2d.show()

# 3D可视化
fig_3d = px.scatter_3d(
    x=emb_3d[:,0],
    y=emb_3d[:,1],
    z=emb_3d[:,2],
    color=book_labels,
    text=valid_persons,
    title="金庸人物关系三维可视化",
    labels={'x': 'PC1', 'y': 'PC2', 'z': 'PC3'},
    hover_name=valid_persons,
    hover_data={'所属书籍': book_labels}
)
fig_3d.update_traces(
    marker=dict(size=5, opacity=0.8),
    textposition='top center'
)
fig_3d.update_layout(height=800)
fig_3d.write_html("person_3d.html")
fig_3d.show()

print("\n可视化完成！图表已保存为person_2d.html和person_3d.html")


可视化完成！图表已保存为person_2d.html和person_3d.html
